In [53]:
import os
import pandas as pd

from cube.api import Cube
from utils.data_transformation import prepare_dialogs, transform_raw_data

In [54]:
# 0) Firstly, use data_transformation.py to get "subdialogs_{id}.csv" files


# 1) change to language of articles
lang = "ua"

DIALOG_IDS = ["138918380", "470409323", "347963763", "850978724"]
frames = []
df_idf = pd.DataFrame()
general_n_subdialogs = 0

for n_id, id in enumerate(DIALOG_IDS):
    path_to_prepared_dialog = os.path.join("data", "all_prepared_dialogs", f"subdialogs_{id}.csv")

    data_for_df = []
    data = pd.read_csv(path_to_prepared_dialog)

    n_subdialog = 0
    dialog_dict, dialog_text = {}, ''
    for row in data.index:
        if n_subdialog != data["subdialog_id"][row]:
            n_subdialog += 1
            general_n_subdialogs += 1
            if n_subdialog != 1:
                dialog_dict["body"] = dialog_text
                data_for_df.append(dialog_dict)

            dialog_dict = {}
            dialog_text = ''
            dialog_dict["id"] = general_n_subdialogs
            dialog_dict["title"] = ""

        if not pd.isnull(data["message"][row]):
            dialog_text += " " + data["message"][row]

    df = pd.DataFrame(data_for_df)
    if n_id == 0:
        df_idf = df
    else:
        df_idf = pd.concat([df_idf, df])

# extend n_rows if you want to analyse keywords more
# df_idf = df_idf[:20]
df_idf

,id,title,body
0,1,,Тому лишилися без улову)) А ми з собою підвод...
1,2,,Тримай) і де результат чекати та і написати к...
2,3,,Mkdir build ..\ncmake -DCMAKE_EXPORT_COMPILE_...
3,4,,Я заберу трохи прикольні принти)) ахахах може...
4,5,,"Я вільний завтра до 12 або після 20 Якщо так,..."
...,...,...,...
38,201,,facebook.com якщый складно хай чий неактивний...
39,202,,встигнуть постараюся завтра закидать ажур мож...
40,203,,git checkout flask app клоновать увести githu...
41,204,,змозить суботу colorlib.com інша команда взят...


In [55]:
# # 1) write your number of rows to analyse. max == 695,
# # but to clean and analyse 30 rows is spending 6 minutes
# path_to_articles = os.path.join('data', 'processed_dialog_files', 'ukr_articles')
# data = pd.read_csv(path_to_articles, sep = ';', header = None)
# df_idf = pd.DataFrame(data[:20])
#
# df_idf.columns = ["id", "title", "en_title", "body", "date", "source_url",
#                   "article_url"]
#
# print("Schema:\n\n",df_idf.dtypes)
# print("Number of questions,columns=",df_idf.shape)
# df_idf

In [56]:

df_idf.dropna(subset = ["title"], inplace=True)
df_idf.dropna(subset = ["body"], inplace=True)

df_idf['text'] = df_idf['title'] + df_idf['body']

# if you have untransformed_data so uncomment it !!!!!!!!!!!!!
# if lang == "ua":
#     cube = Cube(verbose=True)
#     cube.load("uk")
# df_idf['text'] = df_idf['text'].apply(lambda x: transform_raw_data(x, lang, "words_frequency", cube))

#show the first 'text'
df_idf['text'][2]

2     Mkdir build ..\ncmake -DCMAKE_EXPORT_COMPILE_...
2     окей mem errors доступаєшся заброненої пам ят...
2     Скинуть\n- Фейсбук ісф\n- код дурмана групу ​...
2     варіант такой ставалось вручнуть запускать ди...
Name: text, dtype: object

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words():
    """load stop words """

    try:
        with open(os.path.join(os.getcwd(), "..", "dicts", "ukrainian_stopwords.txt"), "r", encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    except FileNotFoundError:
        with open(os.path.join(os.getcwd(), "dicts", "ukrainian_stopwords.txt"), "r",
                  encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    return stop_words

#load a set of stop words
stopwords=get_stop_words()

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of words,
#ignore words that appear in 85% of documents,
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

d:\python\envs\venv_telegram_analysis\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_io', 'analysis', 'dialogs', 'dicts', 'encoding', 'mode', 'my_work', 'name', 'programming', 'telegram', 'telegram_analysis2', 'textiowrapper', 'txt', 'ukrainian_stopwords', 'utf'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [58]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000, ngram_range=(2, 2))
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(202, 10000)

In [59]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# get test docs into a list
docs_test=df_idf['text'].tolist()
docs_title=df_idf['title'].tolist()
docs_body=df_idf['body'].tolist()

In [60]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====


=====Body=====
 Тому лишилися без улову)) А ми з собою підводного ліхтаря не взяли А вчора місяця не було Кальмари реагують на місячне світло Не було кльову😰 Ну і як улов ? Бо я вчора ввечері пішов кальмари ловити Зараз гляну Привіт) Привіт) ну що щось вдалося ? там у words.txt приклади слів які я шукав я закинув на твою гілку Супер, дякую)) це з практикоюб приходить та коли валгріндом багато тестиш просто хотів ще нагадати для майбутнього звільняти пам'ять після себе) І не маю на телефоні Я не люблю зум а чому по силці в зумі не можна без відео ? Давай ? давай в месенджері ? але ти ж в другій країні) зателефонувати ? ок Ой я так не дуже можу зараз посилання скину набираю) Набереш мене чи напишеш? зараз з клангом вожуся та mvsc ) я тут баг у твоєму завданні знайшов а у тебе зараз є трохи часу ? Типу кімната з кухнею Апартамент Поки ще не маю більше) це ви домик знімаєте ? блін, дуже круто а ще фотки скинути можеш) ого Та, вчора пізно ввечері доїхали) Привіт)) привіт 

In [61]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [62]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):

    # get vector for a single document
    curr_vector=tf_idf_vector[i]

    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)


    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
for result in results:
    result2 = {k: v for k, v in sorted(result.items(), key=lambda item: item[1], reverse=True)}
    print(result2)

df.to_csv(os.path.join("data", "processed_dialog_files", "subdialog_keywords.csv"))
df

{'не було': 0.227, 'не маю': 0.181, 'вже на': 0.181, 'ну як': 0.168, 'привіт привіт': 0.136, 'інету бо': 0.09, 'ять після': 0.09, 'які шукав': 0.09, 'як улов': 0.09, 'як тобі': 0.09}
{'типу як': 0.206, 'якийсь readme': 0.137, 'як перевіряти': 0.137, 'щоб усе': 0.137, 'що запускати': 0.137, 'усе перевірити': 0.137, 'типу що': 0.137, 'проект наш': 0.137, 'перевіряти проект': 0.137, 'наш типу': 0.137}
{'pvs studio': 0.476, 'studio log': 0.227, 'давай на': 0.211, 'на 22': 0.199, 'що ти': 0.183, 'їдеш уже': 0.114, 'що написав': 0.114, 'ще трошки': 0.114, 'це перевірю': 0.114, 'хто записував': 0.114}
{'str size_m': 0.204, 'str capacity_m': 0.204, 'size_m from_len': 0.204, 'result return': 0.204, 'if str': 0.204, 'напишеш як': 0.146, 'from_len extra_buffer': 0.146, 'extra_buffer int': 0.146, '11 30': 0.146, 'return result': 0.136}
{'після 13': 0.208, 'від поку': 0.208, 'все ок': 0.208, 'не знаю': 0.193, 'що не': 0.139, 'то пиши': 0.139, 'тебе після': 0.139, 'на 15': 0.139, 'може на': 0.139, '

,doc,keywords
0,Тому лишилися без улову)) А ми з собою підвод...,"{'не було': 0.227, 'не маю': 0.181, 'вже на': ..."
1,Тримай) і де результат чекати та і написати к...,"{'типу як': 0.206, 'якийсь readme': 0.137, 'як..."
2,Mkdir build ..\ncmake -DCMAKE_EXPORT_COMPILE_...,"{'pvs studio': 0.476, 'studio log': 0.227, 'да..."
3,Я заберу трохи прикольні принти)) ахахах може...,"{'str size_m': 0.204, 'str capacity_m': 0.204,..."
4,"Я вільний завтра до 12 або після 20 Якщо так,...","{'після 13': 0.208, 'від поку': 0.208, 'все ок..."
...,...,...
197,facebook.com якщый складно хай чий неактивний...,"{'profession json': 0.29, 'json encoding': 0.2..."
198,встигнуть постараюся завтра закидать ажур мож...,"{'язані колір': 0.296, 'хороший робот': 0.296,..."
199,git checkout flask app клоновать увести githu...,"{'являлася один': 0.191, 'чудово практично': 0..."
200,змозить суботу colorlib.com інша команда взят...,"{'html сторінка': 0.332, 'якої хочий': 0.166, ..."
